In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

In [0]:
%tensorflow_version 1.x
!pip uninstall tensorflow -y
!pip install --upgrade tensorflow-gpu==1.12.2 
!pip install --upgrade keras==2.2.4

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
input_size = np.prod(X_train.shape[1:])
number_of_training_samples = len(X_train)
number_of_testing_samples = len(X_test)

In [0]:
X_train = X_train.reshape(number_of_training_samples, input_size)
X_test = X_test.reshape(number_of_testing_samples, input_size)

X_train = X_train.astype('float32') / 127.5 - 1.
X_test = X_test.astype('float32') / 127.5 - 1.

In [0]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.0002, beta_1=0.5)

In [0]:
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Sequential, Model

from keras.layers.advanced_activations import LeakyReLU

In [0]:
latent_dimension = 100

generator = Sequential()

generator.add(Dense(256, input_dim=latent_dimension))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(input_size, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
discriminator = Sequential()

discriminator.add(Dense(1024, input_dim=input_size,))
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
discriminator.trainable = False
input_images = Input(shape=(latent_dimension, ))
generated_images = generator(input_images)
predictions = discriminator(generated_images)
gan = Model(input_images, predictions)
gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
import matplotlib.pyplot as plot

def plot_loss(losses):

    discriminator_loss = [v[0] for v in losses["discriminator"]]
    generator_loss = [v[0] for v in losses["generator"]]
    
    plot.figure(figsize=(10,8))
    plot.plot(discriminator_loss, label="Discriminator loss")
    plot.plot(generator_loss, label="Generator loss")

    plot.xlabel('Epochs')
    plot.ylabel('Loss')

    plot.legend()
    plot.show()

In [0]:
import matplotlib.pyplot as plot

def plot_generated(number_of_samples=10, dim=(1, 10), figsize=(12, 2)):

    noise = np.random.normal(0, 1, size=(number_of_samples, latent_dimension))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(number_of_samples, 28, 28)

    plot.figure(figsize=figsize)
    for i in range(number_of_samples):
        plot.subplot(dim[0], dim[1], i+1)
        plot.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plot.axis('off')

    plot.tight_layout()
    plot.show()

In [0]:
from tqdm import tqdm_notebook

losses = {"discriminator":[], "generator":[]}

def train(epochs=1, plot_frequency=1, batch_size=128):
    batch_count = int(X_train.shape[0] / batch_size)

    print('Epochs:', epochs)
    print('Batch size:', batch_size)
    print('Batches per epoch:', batch_count)
    
    for epoch in tqdm_notebook(range(1, epochs+1)):

        if (epoch == 1) or (epoch % plot_frequency == 0):
            print('-'*15, 'Epoch %d' % epoch, '-'*15)
            
        for _ in range(batch_count):

            image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

            noise = np.random.normal(0, 1, size=(batch_size, latent_dimension))            
            generated_images = generator.predict(noise)

            X = np.concatenate((image_batch, generated_images))

            y = np.zeros(2*batch_size)
            y[:batch_size] = 0.9

            discriminator.trainable = True
            discriminator_loss = discriminator.train_on_batch(X, y)

            noise = np.random.normal(0, 1, size=(batch_size, latent_dimension))
            y2 = np.ones(batch_size)

            discriminator.trainable = False
            generator_loss = gan.train_on_batch(noise, y2)

        losses["discriminator"].append(discriminator_loss)
        losses["generator"].append(generator_loss)

        if ( epoch == 1 ) or (epoch%plot_frequency == 0):
            plot_generated()

    plot_loss(losses)

In [0]:
train(epochs=200, plot_frequency=20, batch_size=128)